In [1]:
# Import Packages
import pandas as pd
import numpy as np
import plotly.express as px
import re
import ipywidgets as widgets
from IPython.display import display
import json
import plotly.graph_objects as go

file_path = "/Volumes/PKBACK# 001/NuLine/Python Projects - Original/PG10 Procurement Tool v2/PG10 - BOM 08.07.2024 - python copy 2.xlsm"

df_machined = pd.read_excel(file_path, sheet_name="Machined")
df_purchased = pd.read_excel(file_path, sheet_name="Purchased")
df_assys = pd.read_excel(file_path, sheet_name="Assemblies")


df_machined = df_machined[['Machined', 'Description', 'Cost', 'Total Qty',
       'PO QTY', 'Need To Order?', 'QB Qty Rec', 'BP QTY REC', 'OUT TO PLATE',
       'QTY TO PLATE', 'BACK FROM PLATE', 'QTY FROM PLATE', 'Mfg/Matl',
       'Finish', 'Vendor',
       'Order Date', 'Conf Date', 'PO #', 'Locations']]
df_machined['Type'] = 'Machined'
df_purchased = df_purchased[['Purchased', 'Description', 'Cost', 'PK QTY', 'BOM QTY', 'ORDER QTY',
       'PO QTY', 'QB REC QTY', 'ACT REC QTY',
       'Mfg/Matl', 'Vendor', 'Order Date',
       'Conf Date', 'PO #', 'Locations']]
df_purchased['Type'] = 'Purchased'

df_assys = df_assys[['Assy', 'Description', 'Total Qty', 'Locations']]

df_machined.rename(columns={'Machined': 'Part Number'}, inplace=True)
df_purchased.rename(columns={'Purchased': 'Part Number'}, inplace=True)



In [2]:
#combining tables into one
df_all_parts = pd.concat([df_machined, df_purchased], ignore_index=True)
df_all_parts

,Part Number,Description,Cost,Total Qty,PO QTY,Need To Order?,QB Qty Rec,BP QTY REC,OUT TO PLATE,QTY TO PLATE,...,Order Date,Conf Date,PO #,Locations,Type,PK QTY,BOM QTY,ORDER QTY,QB REC QTY,ACT REC QTY
0,Component Spacer.A,Metal Rectangle,0.000,30.0,400.0,,400.0,400.0,NaT,400.0,...,2024-07-10,2024-08-09,55593,", 18x1.3.7.2, 4x4.1.7.3, 4x4.1.8.4, 4x6.10.15,",Machined,NaN,NaN,NaN,NaN,NaN
1,PG10.1.1.1.1.A,M8 Leveling Foot Plate,36.650,38.0,40.0,,40.0,40.0,NaT,40.0,...,2024-06-10,2024-07-17,55545,", 38x1.1.1.1,",Machined,NaN,NaN,NaN,NaN,NaN
2,PG10.1.1.1.A,MODIFIED BRX 90X90 -I- 3410mm,0.000,2.0,2.0,,2.0,2.0,NaT,2.0,...,NaT,NaT,NaN,", 2x1.1.1,",Machined,NaN,NaN,NaN,NaN,NaN
3,PG10.1.1.10.2.A,ACTUATED DANCER FITTINGS PLATE,62.500,2.0,2.0,,2.0,2.0,NaT,2.0,...,2024-05-23,2024-06-24,55523,", 2x1.1.10.1,",Machined,NaN,NaN,NaN,NaN,NaN
4,PG10.1.1.11.1.A,Twin Wall Top Guard 1,84.900,1.0,1.0,,NaN,NaN,NaT,NaN,...,2024-09-19,NaT,55745,", 1x1.1.11.1,",Machined,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977,7000-12481-0000000,"M12 male 0° A-cod. IDC 4-pol., 0.25 - 0.5mm, 4...",0.000,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaT,NaT,NaN,", 1x6.2.1.4.308, 1x6.3.1.4.308,",Purchased,1.0,2.0,2.0,NaN,NaN
978,Zipper Enclosure,Presto Zipper Enclosure,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,2024-01-16,2024-09-26,55366,", 1x12.313",Purchased,1.0,1.0,1.0,NaN,NaN
979,MA-00779,"Presto Turner, Welders, Parkers, Adjustment Me...",0.000,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,2024-01-16,2024-09-26,55366,", 1x3.4.301, 2x4.4.303,",Purchased,1.0,3.0,3.0,NaN,NaN
980,3842535458,BRX GOTO QUICK CONNECTOR N10 0°,4.170,NaN,1750.0,NaN,NaN,NaN,NaT,NaN,...,2024-03-11,2024-03-27,55445,", 127x1.1.302, 4x1.4.302, 4x1.5.302, 56x2.1.30...",Purchased,1.0,1786.0,1786.0,1467.0,1467.0


In [3]:
df_all_parts.columns

Index(['Part Number', 'Description', 'Cost', 'Total Qty', 'PO QTY',
       'Need To Order?', 'QB Qty Rec', 'BP QTY REC', 'OUT TO PLATE',
       'QTY TO PLATE', 'BACK FROM PLATE', 'QTY FROM PLATE', 'Mfg/Matl',
       'Finish', 'Vendor', 'Order Date', 'Conf Date', 'PO #', 'Locations',
       'Type', 'PK QTY', 'BOM QTY', 'ORDER QTY', 'QB REC QTY', 'ACT REC QTY'],
      dtype='object')

In [4]:
# Initialize a list to hold new rows
new_rows = []

# Process each row
for index, row in df_all_parts.iterrows():
    try:
        # Handle locations_value and split based on ', '
        locations_value = row['Locations']
        
        # Ensure locations_value is a string
        locations_value = str(locations_value)
        
        # Split locations_value by ', ' (assuming there are multiple locations)
        locations_list = locations_value.split(', ')
        
        for location in locations_list:
            if 'x' in location:
                # Split by 'x' and take the first part as the quantity
                location_qty, location_rest = location.split('x', 1)
                
                # Extract and process the location
                location_qty = int(location_qty)
                location_rest = location_rest.rsplit('.', 1)[0]  # Remove the last period and characters after
                
                # Build the new row dictionary
                new_row = row.drop(labels=['Locations']).to_dict()
                new_row['Location Qty'] = location_qty
                new_row['Locations'] = location_rest
                
                # Append the new row
                new_rows.append(new_row)
                
    except Exception as e:
        print(f"Error processing row {index}: {e}")

# Create the new DataFrame
df_all_parts_expanded = pd.DataFrame(new_rows)

# Reorder columns to place 'Location Qty' to the left of 'Locations'
# Start by copying the list of columns and inserting 'Location Qty'
ordered_columns = [
    'Part Number', 'Description', 'Cost', 'Total Qty', 'PO QTY',
    'Need To Order?', 'QB Qty Rec', 'BP QTY REC', 'OUT TO PLATE',
    'QTY TO PLATE', 'BACK FROM PLATE', 'QTY FROM PLATE', 'Mfg/Matl',
    'Finish', 'Vendor', 'Order Date', 'Conf Date', 'PO #', 'Location Qty',
    'Locations', 'Type', 'PK QTY', 'BOM QTY', 'ORDER QTY', 'QB REC QTY', 'ACT REC QTY'
]

df_all_parts_expanded = df_all_parts_expanded[ordered_columns]

print(df_all_parts_expanded)

             Part Number                      Description    Cost  Total Qty  \
0     Component Spacer.A                  Metal Rectangle   0.000       30.0   
1     Component Spacer.A                  Metal Rectangle   0.000       30.0   
2     Component Spacer.A                  Metal Rectangle   0.000       30.0   
3     Component Spacer.A                  Metal Rectangle   0.000       30.0   
4         PG10.1.1.1.1.A           M8 Leveling Foot Plate  36.650       38.0   
...                  ...                              ...     ...        ...   
1771          3842535458  BRX GOTO QUICK CONNECTOR N10 0°   4.170        NaN   
1772          3842535458  BRX GOTO QUICK CONNECTOR N10 0°   4.170        NaN   
1773          3842535458  BRX GOTO QUICK CONNECTOR N10 0°   4.170        NaN   
1774          3842535458  BRX GOTO QUICK CONNECTOR N10 0°   4.170        NaN   
1775             F66BK60            6x6in wireway divider  66.541        NaN   

      PO QTY Need To Order?  QB Qty Rec

In [5]:
#Status processing, removed statements that were overwriting previous statuses in the case of extras 
# Add a new 'Status' column with default value
df_all_parts_expanded['Status'] = ''

# Function to update Status based on the type and quantities
def update_status(group):
    if group['Type'].iloc[0] == 'Machined':
        qty_from_plate = group['QTY FROM PLATE'].iloc[0]
        for i in range(len(group)):
            location_qty = group['Location Qty'].iloc[i]
            if qty_from_plate > 0:
                if qty_from_plate >= location_qty:
                    group.at[group.index[i], 'Status'] = 'Received'
                    qty_from_plate -= location_qty
                else:
                    group.at[group.index[i], 'Status'] = f'Received {int(qty_from_plate)} of {int(location_qty)}'
                    qty_from_plate = 0
            else:
                group.at[group.index[i], 'Status'] = 'Not Received'
    elif group['Type'].iloc[0] == 'Purchased':
        act_rec_qty = group['ACT REC QTY'].iloc[0]
        for i in range(len(group)):
            location_qty = group['Location Qty'].iloc[i]
            if act_rec_qty > 0:
                if act_rec_qty >= location_qty:
                    group.at[group.index[i], 'Status'] = 'Received'
                    act_rec_qty -= location_qty
                else:
                    group.at[group.index[i], 'Status'] = f'Received {int(act_rec_qty)} of {int(location_qty)}'
                    act_rec_qty = 0
            else:
                group.at[group.index[i], 'Status'] = 'Not Received'
    return group

# Apply the update_status function to each group of 'Part Number'
df_all_parts_expanded = df_all_parts_expanded.groupby('Part Number').apply(update_status)

print(df_all_parts_expanded)

                            Part Number                   Description   Cost  \
Part Number                                                                    
22198            1144             22198                  Dienes Knife  20.21   
42027            1540             42027  BPM 24 Icotek Blanking Plate   6.72   
                 1541             42027  BPM 24 Icotek Blanking Plate   6.72   
                 1542             42027  BPM 24 Icotek Blanking Plate   6.72   
                 1543             42027  BPM 24 Icotek Blanking Plate   6.72   
...                                 ...                           ...    ...   
WS-10-30-1400    1195     WS-10-30-1400                   LINIAR RAIL  48.79   
WW-10-30-08      1196       WW-10-30-08            LINER DRYLIN STAGE  88.84   
Y600-D           818             Y600-D               Airprep Spacer    6.52   
Y600T-D          846            Y600T-D   Airprep Spacer with Bracket   9.00   
Zipper Enclosure 1738  Zipper Enclosure 

/var/folders/0w/q_gsy2xs38s4d9955d8s3gkc0000gn/T/ipykernel_55545/3291768520.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_all_parts_expanded = df_all_parts_expanded.groupby('Part Number').apply(update_status)


# new assy status that will calculate weighted data based on total parts of each type in each location as well as percentages received

In [6]:
def calculate_assembly_status(df):
    # Ensure that 'Locations' is the correct column name
    if 'Locations' not in df.columns:
        raise ValueError("Column 'Locations' not found in the DataFrame")
    
    # Group by location
    grouped = df.groupby('Locations')

    # Dictionary to hold the results
    results = []

    for location, group in grouped:
        # Separate Machined and Purchased
        machined_parts = group[group['Type'] == 'Machined']
        purchased_parts = group[group['Type'] == 'Purchased']

        # Calculate total unique machined parts
        total_unique_machined = len(machined_parts['Part Number'].unique())
        
        # Calculate total machined parts
        total_machined = machined_parts['Location Qty'].sum()
        if total_unique_machined == 0:
            machined_received_percent = 100
            not_received_machined_str = "No Machined Parts"
        else:
            received_machined = len(machined_parts[machined_parts['Status'] == 'Received']['Part Number'].unique())
            machined_received_percent = (received_machined / total_unique_machined) * 100
            not_received_machined = list(machined_parts[machined_parts['Status'] != 'Received']['Part Number'].unique())
            not_received_machined_str = ', '.join(map(str, not_received_machined))

        # Calculate total unique purchased parts
        total_unique_purchased = len(purchased_parts['Part Number'].unique())
        
        # Calculate total purchased parts
        total_purchased = purchased_parts['Location Qty'].sum()
        if total_unique_purchased == 0:
            purchased_received_percent = 100
            not_received_purchased_str = "No Purchased Parts"
        else:
            received_purchased = len(purchased_parts[purchased_parts['Status'] == 'Received']['Part Number'].unique())
            partially_received_purchased = len(purchased_parts[purchased_parts['Status'] == 'Partially Received']['Part Number'].unique())
            purchased_received_percent = (received_purchased / total_unique_purchased) * 100
            not_received_purchased = list(purchased_parts[purchased_parts['Status'] != 'Received']['Part Number'].unique())
            not_received_purchased_str = ', '.join(map(str, not_received_purchased))

        # Append results
        results.append({
            'Locations': location,
            'Total Unique Machined': total_unique_machined,
            'Total Machined': total_machined,
            'Machined Received %': f"{machined_received_percent:.0f}%",
            'Machined Not Received': not_received_machined_str,
            'Total Unique Purchased': total_unique_purchased,
            'Total Purchased': total_purchased,
            'Purchased Received %': f"{purchased_received_percent:.0f}%",
            'Purchased Not Received': not_received_purchased_str
        })

    # Create DataFrame from results
    df_assembly_status = pd.DataFrame(results)
    return df_assembly_status

df_assembly_status = calculate_assembly_status(df_all_parts_expanded)
print(df_assembly_status)

    Locations  Total Unique Machined  Total Machined Machined Received %  \
0         0.1                      0               0                100%   
1        0.10                      0               0                100%   
2        0.11                      0               0                100%   
3         0.2                      0               0                100%   
4         0.3                      0               0                100%   
..        ...                    ...             ...                 ...   
247       9.1                      6               7                100%   
248       9.2                      0               0                100%   
249       9.3                      3               4                100%   
250       9.4                      3              16                100%   
251       9.5                      2               8                100%   

    Machined Not Received  Total Unique Purchased  Total Purchased  \
0       No Machin

# working with assys dataframe

In [7]:
#processing the assys dataframe

# Step 1: Expand rows by location
new_rows = []

for index, row in df_assys.iterrows():
    locations_value = row['Locations']
    if pd.isna(locations_value):
        locations_value = ''
        
    locations_list = str(locations_value).split(', ')
    
    for location in locations_list:
        new_row = row.drop(labels=['Locations']).to_dict()
        new_row['Locations'] = location
        new_rows.append(new_row)

# Create a new DataFrame from the list of new rows
df_assys_expanded = pd.DataFrame(new_rows)

# Debug: Print the columns of the expanded DataFrame
print("Columns in df_assys_expanded:", df_assys_expanded.columns)

# Step 2: Process the Locations column
def process_locations(locations):
    locations = locations.strip()  # Remove any leading/trailing spaces
    if 'x' in locations:
        location_qty, location_rest = locations.split('x', 1)
        try:
            location_qty = int(location_qty)  # Convert the number before 'x' to integer
        except ValueError:
            location_qty = 0  # Handle cases where conversion fails
        location_rest = location_rest.rsplit('.', 1)[0]  # Remove the last period and characters after
        return pd.Series([location_qty, location_rest])
    else:
        return pd.Series([0, locations])  # Handle cases where 'x' is missing

# Apply the function and create new columns
df_assys_expanded[['Assy Location Qty', 'Locations']] = df_assys_expanded['Locations'].apply(process_locations)

# Debug: Print the DataFrame after processing
print(df_assys_expanded)

# Step 3: Remove invalid rows except for outlier
# Define a mask for valid rows
valid_mask = (df_assys_expanded['Locations'].str.strip() != '') | (df_assys_expanded['Assy'] == '0')

# Apply the mask to filter the DataFrame
df_filtered_assys = df_assys_expanded[valid_mask]

# For rows with assy number '0', clear the 'Locations' column if it's invalid
df_filtered_assys.loc[df_filtered_assys['Assy'] == '0', 'Locations'] = df_filtered_assys.loc[df_filtered_assys['Assy'] == '0', 'Locations'].apply(lambda x: '' if x.strip() in [',', ''] else x)

#rearrange columns? this didnt work below
#df_filtered_assys = df_filtered_assys['Assy', 'Description', 'Total Qty', 'Assy Location Qty', 'Locations']

# Debug: Print the filtered DataFrame
print(df_filtered_assys)

Columns in df_assys_expanded: Index(['Assy', 'Description', 'Total Qty', 'Locations'], dtype='object')
     Assy                 Description  Total Qty Locations  Assy Location Qty
0       0          2-Up Pouch Machine          0         ,                  0
1     0.1              Main Enclosure          1                            0
2     0.1              Main Enclosure          1        12                  1
3     0.2        Unwind HMI Enclosure          1                            0
4     0.2        Unwind HMI Enclosure          1         1                  1
..    ...                         ...        ...       ...                ...
715  11.2  Air Supply Pipe Mount Assy         10                            0
716  11.2  Air Supply Pipe Mount Assy         10        11                 10
717  11.3     Air Supply Pipe Tie Off         24                            0
718  11.3     Air Supply Pipe Tie Off         24        11                 24
719    12     Plinth and Wireway Assy  

# merging the two dataframes

In [8]:
def merge_and_adjust_columns(df_filtered_assys, df_assembly_status):
    # Ensure 'Assy' and 'Locations' columns are present
    if 'Assy' not in df_filtered_assys.columns:
        raise ValueError("Column 'Assy' not found in df_filtered_assys")
    if 'Locations' not in df_assembly_status.columns:
        raise ValueError("Column 'Locations' not found in df_assembly_status")
    
    # Rename 'Locations' column in df_filtered_assys to 'Assy Locations'
    df_filtered_assys = df_filtered_assys.rename(columns={'Locations': 'Assy Locations'})
    
    # Rename 'Locations' column in df_assembly_status to 'Part Locations'
    df_assembly_status = df_assembly_status.rename(columns={'Locations': 'Part Locations'})
    
    # Perform the merge to get the 'Part Locations' and the new columns from df_assembly_status
    columns_to_merge = [
        'Part Locations', 'Total Unique Machined', 'Total Machined',
        'Machined Received %', 'Machined Not Received',
        'Total Unique Purchased', 'Total Purchased',
        'Purchased Received %', 'Purchased Not Received'
    ]
    
    merged_df = pd.merge(df_filtered_assys, df_assembly_status[columns_to_merge], 
                         left_on='Assy', right_on='Part Locations', how='left')
    
    # Drop the original 'Assy Locations' column from merged_df if needed, or leave it as is
    # (You can choose to drop it if it is redundant)
    # merged_df = merged_df.drop(columns=['Assy Locations'])
    
    return merged_df

merged_df = merge_and_adjust_columns(df_filtered_assys, df_assembly_status)
display(merged_df)

,Assy,Description,Total Qty,Assy Locations,Assy Location Qty,Part Locations,Total Unique Machined,Total Machined,Machined Received %,Machined Not Received,Total Unique Purchased,Total Purchased,Purchased Received %,Purchased Not Received
0,0,2-Up Pouch Machine,0,,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.1,Main Enclosure,1,12,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.2,Unwind HMI Enclosure,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.3,Unwind IO Enclosure,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.3.1,Manifold 1 Unwind Nips,1,0.3,1,0.3.1,0.0,0.0,100%,No Machined Parts,1.0,1.0,100%,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,11,Air Prep Assy,1,0,1,11,0.0,0.0,100%,No Machined Parts,7.0,15.0,100%,
456,11.1,Air Prep Station,1,11,1,11.1,1.0,1.0,100%,,12.0,16.0,75%,"4464K44, 4464K892, FWSSM-D20-V10.5-T8"
457,11.2,Air Supply Pipe Mount Assy,10,11,10,11.2,0.0,0.0,100%,No Machined Parts,2.0,20.0,50%,3842530287
458,11.3,Air Supply Pipe Tie Off,24,11,24,11.3,0.0,0.0,100%,No Machined Parts,2.0,48.0,0%,"KQ2H10-U04A, PPS1 CI63"


In [9]:
# sorting merged_df

def parse_assy_location(location):
    # Convert Assy Locations to sortable tuples
    if isinstance(location, str) and location:
        return tuple(map(int, location.split('.')))
    return ()

def parse_assy(assy):
    # Convert Assy to sortable tuples
    if isinstance(assy, str):
        return tuple(map(int, assy.split('.')))
    return ()

# Convert columns to sortable tuples
merged_df['Sortable Assy Locations'] = merged_df['Assy Locations'].apply(parse_assy_location)
merged_df['Sortable Assy'] = merged_df['Assy'].apply(parse_assy)

# Sort the DataFrame first by 'Sortable Assy Locations' and then by 'Sortable Assy'
sorted_merged_df = merged_df.sort_values(by=['Sortable Assy Locations', 'Sortable Assy']).drop(columns=['Sortable Assy Locations', 'Sortable Assy'])

display(sorted_merged_df)
print(sorted_merged_df)

,Assy,Description,Total Qty,Assy Locations,Assy Location Qty,Part Locations,Total Unique Machined,Total Machined,Machined Received %,Machined Not Received,Total Unique Purchased,Total Purchased,Purchased Received %,Purchased Not Received
0,0,2-Up Pouch Machine,0,,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,1,Unwind Assy,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,2,Main Frame 1,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,3,Main Frame 2,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227,4,Main Frame 3,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,10.3.1,CONVEYOR L BRACKET ASSY,4,10.3,4,10.3.1,1.0,4.0,100%,,2.0,20.0,50%,201558
456,11.1,Air Prep Station,1,11,1,11.1,1.0,1.0,100%,,12.0,16.0,75%,"4464K44, 4464K892, FWSSM-D20-V10.5-T8"
457,11.2,Air Supply Pipe Mount Assy,10,11,10,11.2,0.0,0.0,100%,No Machined Parts,2.0,20.0,50%,3842530287
458,11.3,Air Supply Pipe Tie Off,24,11,24,11.3,0.0,0.0,100%,No Machined Parts,2.0,48.0,0%,"KQ2H10-U04A, PPS1 CI63"


       Assy                 Description  Total Qty Assy Locations  \
0         0          2-Up Pouch Machine          0                  
16        1                 Unwind Assy          1              0   
146       2                Main Frame 1          1              0   
198       3                Main Frame 2          1              0   
227       4                Main Frame 3          1              0   
..      ...                         ...        ...            ...   
454  10.3.1     CONVEYOR L BRACKET ASSY          4           10.3   
456    11.1            Air Prep Station          1             11   
457    11.2  Air Supply Pipe Mount Assy         10             11   
458    11.3     Air Supply Pipe Tie Off         24             11   
1       0.1              Main Enclosure          1             12   

     Assy Location Qty Part Locations  Total Unique Machined  Total Machined  \
0                    0            NaN                    NaN             NaN   
16         

In [10]:
sorted_merged_df['Assy'] = sorted_merged_df['Assy'].astype(str)
sorted_merged_df['Assy Locations'] = sorted_merged_df['Assy Locations'].astype(str)

# Composite Received Calc

In [11]:
# copy the sorted_merged_df to work with it
df_composite = sorted_merged_df.copy()

# Remove '%' sign and convert percentage columns to numeric values
df_composite['Machined Received %'] = df_composite['Machined Received %'].str.rstrip('%').astype(float)
df_composite['Purchased Received %'] = df_composite['Purchased Received %'].str.rstrip('%').astype(float)

# Fill NaN with 0s for calculation purposes
df_composite['Total Unique Machined'] = df_composite['Total Unique Machined'].fillna(0)
df_composite['Total Unique Purchased'] = df_composite['Total Unique Purchased'].fillna(0)
df_composite['Machined Received %'] = df_composite['Machined Received %'].fillna(0)
df_composite['Purchased Received %'] = df_composite['Purchased Received %'].fillna(0)

# Calculate Composite Received Percentage
total_unique_parts = df_composite['Total Unique Machined'] + df_composite['Total Unique Purchased']

df_composite['Composite Received %'] = (
    (df_composite['Total Unique Machined'] * df_composite['Machined Received %'] +
     df_composite['Total Unique Purchased'] * df_composite['Purchased Received %']) / total_unique_parts
).fillna(0)

# Concatenate 'Machined Not Received' and 'Purchased Not Received'
df_composite['Composite Not Received'] = df_composite['Machined Not Received'].fillna('') + ', ' + df_composite['Purchased Not Received'].fillna('')
df_composite['Composite Not Received'] = df_composite['Composite Not Received'].str.strip(', ')

df_composite['Total Parts'] = df_composite['Total Unique Machined'] + df_composite['Total Unique Purchased']

# Display the entire updated DataFrame
print(df_composite)

       Assy                 Description  Total Qty Assy Locations  \
0         0          2-Up Pouch Machine          0                  
16        1                 Unwind Assy          1              0   
146       2                Main Frame 1          1              0   
198       3                Main Frame 2          1              0   
227       4                Main Frame 3          1              0   
..      ...                         ...        ...            ...   
454  10.3.1     CONVEYOR L BRACKET ASSY          4           10.3   
456    11.1            Air Prep Station          1             11   
457    11.2  Air Supply Pipe Mount Assy         10             11   
458    11.3     Air Supply Pipe Tie Off         24             11   
1       0.1              Main Enclosure          1             12   

     Assy Location Qty Part Locations  Total Unique Machined  Total Machined  \
0                    0            NaN                    0.0             NaN   
16         

# 3rd try, adding tier column to treemap code

In [12]:
# Example: starting from your DataFrame
df_temp_hierarchy = df_composite.copy()

def build_hierarchy(df, parent_assy):
    children = df[df['Assy Locations'] == parent_assy]
    hierarchy = {}
    for _, row in children.iterrows():
        child_assy = row['Assy']
        hierarchy[row['Description']] = {
            'Total Unique Machined': row.get('Total Unique Machined', 0),
            'Total Unique Purchased': row.get('Total Unique Purchased', 0),
            'Composite Received %': row.get('Composite Received %', 0),
            'Composite Not Received': row.get('Composite Not Received', ''),
            'Children': build_hierarchy(df, child_assy)  # Recursive call
        }
    return hierarchy

# Build the hierarchy starting from the top level (Assy = '0')
treemap_data = build_hierarchy(df_temp_hierarchy, '0')

def flatten_hierarchy(hierarchy, parent_name=""):
    flat_data = []
    for name, data in hierarchy.items():
        flat_name = f"{parent_name} / {name}" if parent_name else name
        total_unique_parts = data['Total Unique Machined'] + data['Total Unique Purchased']
        flat_data.append({
            'path': flat_name,
            'Total Unique Parts': total_unique_parts,
            'Composite Received %': data.get('Composite Received %', 0),
            'Composite Not Received': data.get('Composite Not Received', '')
        })
        flat_data.extend(flatten_hierarchy(data['Children'], flat_name))
    return flat_data

flat_data = flatten_hierarchy(treemap_data)
treemap_df = pd.DataFrame(flat_data)

# Add a 'Tier' column by counting the number of '/' and adding 1
#treemap_df['Tier'] = treemap_df['path'].apply(lambda x: x.count('/') + 1)

# Display the updated DataFrame
display(treemap_df)


,path,Total Unique Parts,Composite Received %,Composite Not Received
0,Unwind Assy,0.0,0.000000,
1,Unwind Assy / Unwind HMI Enclosure,0.0,0.000000,
2,Unwind Assy / Unwind IO Enclosure,0.0,0.000000,
3,Unwind Assy / Unwind IO Enclosure / Manifold 1...,1.0,100.000000,No Machined Parts
4,Unwind Assy / Unwind Frame,5.0,100.000000,
...,...,...,...,...
472,Air Prep Assy / Air Prep Station,13.0,76.923077,"4464K44, 4464K892, FWSSM-D20-V10.5-T8"
473,Air Prep Assy / Air Supply Pipe Mount Assy,2.0,50.000000,"No Machined Parts, 3842530287"
474,Air Prep Assy / Air Supply Pipe Tie Off,2.0,0.000000,"No Machined Parts, KQ2H10-U04A, PPS1 CI63"
475,Plinth and Wireway Assy,23.0,30.521739,"PG10.12.1.A, PG10.12.10.A, PG10.12.2.A, PG10.1..."


In [13]:
# directly adding in total parts and adding back in the assy location qty""PG10 - BOM 08.07.2024 - python copy 2.xlsm"

# Example: starting from your DataFrame
df_temp_hierarchy = df_composite.copy()

def build_hierarchy(df, parent_assy):
    children = df[df['Assy Locations'] == parent_assy]
    hierarchy = {}
    for _, row in children.iterrows():
        child_assy = row['Assy']
        hierarchy[row['Description']] = {
            'Total Parts': row.get('Total Parts', 0),  # Read 'Total Parts' directly
            'Assy Location Qty': row.get('Assy Location Qty', 0),  # Read 'Assy Location Qty' directly
            'Composite Received %': row.get('Composite Received %', 0),
            'Composite Not Received': row.get('Composite Not Received', ''),
            'Children': build_hierarchy(df, child_assy)  # Recursive call
        }
    return hierarchy

# Build the hierarchy starting from the top level (Assy = '0')
treemap_data = build_hierarchy(df_temp_hierarchy, '0')

def flatten_hierarchy(hierarchy, parent_name=""):
    flat_data = []
    for name, data in hierarchy.items():
        flat_name = f"{parent_name} / {name}" if parent_name else name
        flat_data.append({
            'path': flat_name,
            'Assy Location Qty': data.get('Assy Location Qty', 0),  # Include 'Assy Location Qty'
            'Total Unique Parts': data.get('Total Parts', 0),  # Use 'Total Parts' as 'Total Unique Parts'
            'Composite Received %': data.get('Composite Received %', 0),
            'Composite Not Received': data.get('Composite Not Received', '')
        })
        flat_data.extend(flatten_hierarchy(data['Children'], flat_name))
    return flat_data

flat_data = flatten_hierarchy(treemap_data)
treemap_df = pd.DataFrame(flat_data)

# Add a 'Tier' column by counting the number of '/' and adding 1
# treemap_df['Tier'] = treemap_df['path'].apply(lambda x: x.count('/') + 1)

# Display the updated DataFrame
display(treemap_df)

,path,Assy Location Qty,Total Unique Parts,Composite Received %,Composite Not Received
0,Unwind Assy,1,0.0,0.000000,
1,Unwind Assy / Unwind HMI Enclosure,1,0.0,0.000000,
2,Unwind Assy / Unwind IO Enclosure,1,0.0,0.000000,
3,Unwind Assy / Unwind IO Enclosure / Manifold 1...,1,1.0,100.000000,No Machined Parts
4,Unwind Assy / Unwind Frame,1,5.0,100.000000,
...,...,...,...,...,...
472,Air Prep Assy / Air Prep Station,1,13.0,76.923077,"4464K44, 4464K892, FWSSM-D20-V10.5-T8"
473,Air Prep Assy / Air Supply Pipe Mount Assy,10,2.0,50.000000,"No Machined Parts, 3842530287"
474,Air Prep Assy / Air Supply Pipe Tie Off,24,2.0,0.000000,"No Machined Parts, KQ2H10-U04A, PPS1 CI63"
475,Plinth and Wireway Assy,1,23.0,30.521739,"PG10.12.1.A, PG10.12.10.A, PG10.12.2.A, PG10.1..."


In [14]:
# Function to filter out rows that have children
def filter_out_parent_rows(df, columns_to_keep):
    # Sort by path length to ensure parent rows come before child rows
    df['path_length'] = df['path'].apply(lambda x: len(x.split(' / ')))
    # Identify rows that have children
    df['has_children'] = df['path'].apply(lambda x: any(df['path'].str.startswith(x + ' / ')))
    # Keep rows that don't have children
    filtered_df = df[~df['has_children']].drop(columns=['path_length', 'has_children'])
    
    # Split the path column
    path_columns = filtered_df['path'].str.split(' / ', expand=True)
    path_columns.columns = ['First', 'Second', 'Third', 'Fourth']
    
    # Add the 'Top' column
    path_columns.insert(0, 'Top', '2-Up Pouch Machine')
    
    # Keep the specified original columns
    for col in columns_to_keep:
        if col in df.columns:
            path_columns[col] = filtered_df[col]
    
    return path_columns

# Apply the function
columns_to_keep = ['Assy Location Qty', 'Total Unique Parts', 'Composite Received %', 'Composite Not Received']  # Add the names of the columns you want to keep
filtered_df = filter_out_parent_rows(treemap_df, columns_to_keep)

# Display the filtered data
display(filtered_df)


,Top,First,Second,Third,Fourth,Assy Location Qty,Total Unique Parts,Composite Received %,Composite Not Received
1,2-Up Pouch Machine,Unwind Assy,Unwind HMI Enclosure,None,None,1,0.0,0.000000,
3,2-Up Pouch Machine,Unwind Assy,Unwind IO Enclosure,Manifold 1 Unwind Nips,None,1,1.0,100.000000,No Machined Parts
5,2-Up Pouch Machine,Unwind Assy,Unwind Frame,M8 Leveling Foot Assy,None,10,4.0,75.250000,90591A220
6,2-Up Pouch Machine,Unwind Assy,Unwind Frame,Single Web Roller Assy,None,10,7.0,100.000000,
7,2-Up Pouch Machine,Unwind Assy,Unwind Frame,Frame Gusset,None,16,2.0,50.000000,3842528718
...,...,...,...,...,...,...,...,...,...
470,2-Up Pouch Machine,Delivery Assy,Indexing Conveyor Assemb,CONVEYOR L BRACKET ASSY,None,4,3.0,66.666667,201558
472,2-Up Pouch Machine,Air Prep Assy,Air Prep Station,None,None,1,13.0,76.923077,"4464K44, 4464K892, FWSSM-D20-V10.5-T8"
473,2-Up Pouch Machine,Air Prep Assy,Air Supply Pipe Mount Assy,None,None,10,2.0,50.000000,"No Machined Parts, 3842530287"
474,2-Up Pouch Machine,Air Prep Assy,Air Supply Pipe Tie Off,None,None,24,2.0,0.000000,"No Machined Parts, KQ2H10-U04A, PPS1 CI63"


In [16]:
import plotly.express as px
import numpy as np

# Replace NaN values with 0 in 'Total Unique Parts' column
filtered_df['Total Unique Parts'] = filtered_df['Total Unique Parts'].fillna(0)

# Ensure 'Total Unique Parts' has at least some non-zero values
if filtered_df['Total Unique Parts'].sum() == 0:
    print("Warning: All values in 'Total Unique Parts' are zero. Adding a small value to avoid division by zero.")
    filtered_df['Total Unique Parts'] = filtered_df['Total Unique Parts'] + 1

# Replace NaN values with 0 in 'Composite Received %' column
filtered_df['Composite Received %'] = filtered_df['Composite Received %'].fillna(0)

# Create the treemap
fig = px.treemap(filtered_df, 
                 path=['Top', 'First', 'Second', 'Third', 'Fourth'], 
                 values='Total Unique Parts',
                 #color='Composite Received %', 
                 hover_data=['Composite Not Received'],
                 width=2400, 
                 height=1200
)
fig.update_traces(root_color="lightgrey")
fig.update_layout(
    margin=dict(t=50, l=25, r=25, b=25)
    #uniformtext=dict(minsize=6, mode='show')
    )

fig.show()